In [1]:
import sys
from PyQt4.QtGui import *
from PyQt4.QtCore import *
from PyQt4.QAxContainer import *
import pandas as pd
from Kiwoom import Kiwoom
import time
import datetime

In [2]:
class PyMon:
    def __init__(self):
        self.kiwoom = Kiwoom()
        self.kiwoom.login()
        self.get_code_list()

    # 코스피/코스닥 종목 코드 얻어오기
    def get_code_list(self):
        self.kospi_codes = self.kiwoom.GetCodeListByMarket(0) # KOSPI
        self.kosdak_codes = self.kiwoom.GetCodeListByMarket(10) # KOSDAK
           
    # 일별 데이터 가져오기
    def get_ohlcv(self, code, start_date):
        # Init data structure
        self.kiwoom.InitOHLCVRawData()
        
        # Request TR and get data
        self.kiwoom.SetInputValue("종목코드", code)
        self.kiwoom.SetInputValue("기준일자", start_date)
        self.kiwoom.SetInputValue("수정주가구분", 1)
        self.kiwoom.CommRqData("주식일봉차트조회요청", "opt10081", 0, "0101")
        time.sleep(0.2) # TR 요청이 0.2초 간격 주기 
        
        df = pd.DataFrame(self.kiwoom.ohlc, columns=['open', 'high', 'low', 'close', 'volume'], index=self.kiwoom.ohlc['date'])
        
        return df
    
    # 급등주 포착 알고리즘 
    # : 특정 거래일의 거래량이 이전 시점의 평균 거래량보다 1000% 이상 급증하는 종목을 찾는 알고리즘
    def check_speedy_rising_volume(self, code):
        today = datetime.datetime.today().strftime("%Y%m%d") # 조회 시작일 : 오늘
        # init data structure
        df = self.get_ohlcv(code, today) 
        volumes = df['volume']
        
        sum_vol20 = 0 # 20일 총 거래량
        avg_vol20 = 0 # 20일 평균 거래량
        
        # check small trading days
        # 데이터가 20일이 안되면 False 반환
        if len(volumes) < 21:
            return False
        
        # Accumulation (조회 시작일을 기준으로 이전 20일 간의 거래량 합산)
        for i, vol in enumerate(volumes):
            if i == 0:
                today_vol = vol
            elif i >= 1 and i <= 20 :
                sum_vol20 += vol
            elif i >= 21:
                break
                
        # Average and decision
        avg_vol20 = sum_vol20 / 20; 
        # 급등주인 경우 (조회 시작일의 거래량이 평균 거래량을 1000% 초과할 때) True 반환
        if today_vol > avg_vol20 * 10:
            print(code + ": 급등주 o")
            return True
        else :
            print(code + ": 급등주 x")
            return False
        
    def update_buy_list(self, buy_list):
        
        if (len(buy_list) == 0):
            print("급등주 없음")
        else :
            print("급등주 개수 : " + str(len(buy_list)))
        f = open("buy_list.txt", "wt")
        for code in buy_list:
            f.writelines("매수;%s;시장가;10;0;매수전\n" % (code))
        f.close()

In [3]:
app = QApplication(sys.argv)

In [4]:
pymon = PyMon()

< 로그인 성공 >
----- 사용자 정보 -----
아이디 : youkcc
이름 : 이여진
계좌 번호
8088938611
----------------------


In [5]:
# 1. 코스피, 코스닥 종목 얻어오기
pymon.get_code_list()
print(len(pymon.kospi_codes))
print(pymon.kospi_codes[0:4])
print(len(pymon.kosdak_codes))
print(pymon.kosdak_codes[0:4])

['900050', '000020', '000030', '000040']
['900090', '900080', '900110', '900130']


In [6]:
# 2. 일별 데이터 가져오기
print(pymon.get_ohlcv("039490", "20170315"))

<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

           open   high    low  close  volume
20170315  80000  81500  79500  80400   33167
20170314  79700  81800  78900  80400   59060
20170313  76100  80000  75800  79200   52802
20170310  74400  77300  73300  76800   68185
20170309  75200  76000  74200  75100   34886
20170308  75000  76000  74000  74900   43264
20170307  73300  75700  73300  75500   37741
20170306  73600  74400  72000  74000   46472
20170303  77300  77300  73800  74000   46495
20170302  77900  78900  77000  78000   49709
20170228  76500  77900  75400  77100   60869
20170227  77100  77400  75500  76100   35850
20170224  76900  77900  76000  76300   57565
20170223  78100  78100  76000  77000   60117
20170222  76600  78100  76300  77300   49600
20170221  73500  76900  73300  76500   46682
20170220  74600  75000  73600  73800   30275
20170217  74600  75900  73900  75600   30195
20170216  72100  75900  72100  75800   67704
20170215  71700  73600  71400  718

In [8]:
pymon.kospi_codes

['900090',
 '900080',
 '900110',
 '900130',
 '900180',
 '900270',
 '900260',
 '900290',
 '900300',
 '000250',
 '000440',
 '001000',
 '001540',
 '001810',
 '001840',
 '002230',
 '002290',
 '002680',
 '002800',
 '003100',
 '003310',
 '003670',
 '003800',
 '004590',
 '004650',
 '004780',
 '005160',
 '005290',
 '005670',
 '005710',
 '005860',
 '005990',
 '006050',
 '006140',
 '006580',
 '006730',
 '006910',
 '006920',
 '007330',
 '007370',
 '007390',
 '007530',
 '007720',
 '007770',
 '007820',
 '008290',
 '008370',
 '008470',
 '008800',
 '008830',
 '009300',
 '009520',
 '009620',
 '009730',
 '009780',
 '010170',
 '010240',
 '010280',
 '010470',
 '011040',
 '011080',
 '011320',
 '011370',
 '011560',
 '012340',
 '012620',
 '012700',
 '012790',
 '012860',
 '013030',
 '013120',
 '013310',
 '013720',
 '013810',
 '013990',
 '014100',
 '014190',
 '014200',
 '014470',
 '014570',
 '014620',
 '014940',
 '014970',
 '015710',
 '015750',
 '016100',
 '016170',
 '016250',
 '016600',
 '016670',
 '016920',

In [9]:
# 3. 급등주 종목 찾기
buy_list = []
num = len(pymon.kospi_codes)
for code in pymon.kospi_codes:
    if pymon.check_speedy_rising_volume(code):
        buy_list.append(code)
pymon.update_buy_list(buy_list)

<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 578

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회명 : 주식일봉차트조회요청
레코드 반복 횟수 : 900

No
<< OnReceiveTrData 이벤트 발생 >>
TR 조회